In [18]:
import asyncio
import nest_asyncio
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from IPython.display import clear_output



In [29]:
# Configuración del modelo de lenguaje y plantilla de RAG
model = ChatOllama(model="smollm2:latest")  # Cambiado a llama2 por compatibilidad

RAG_TEMPLATE = """
Eres un asiste de preguntas y respuestas. Usa la informacion recuperada para contestar las preguntas, relacionadas al CADS. Si no sabes la respuesta, solo di que no conoces la respuesta. Usa tres sentencias como maximo para dar una respuesta concisa.

<context>
{context}
</context>

Contesta la siguiente pregunta: 

{question}"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

# Definir la función para formatear documentos
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retriever = vectorstore.as_retriever()

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | model
    | StrOutputParser()
)

# Función para manejar una sola pregunta
async def process_question(question):
    print(f"Procesando pregunta: {question}\n")
    response = ""
    try:
        async for chunk in chain.astream(question):
            response += chunk
            clear_output(wait=True)
            print(f"Pregunta: {question}")
            print(f"Respuesta: {response}", flush=True)
        return response
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

# Función para ejecutar la pregunta
def run_single_question(question):
    return asyncio.get_event_loop().run_until_complete(process_question(question))

In [30]:
respuesta=run_single_question("Cual es la mision del CADS")

Pregunta: Cual es la mision del CADS
Respuesta: <response>
El CADS tiene una misión claramente definida: asegurarse de brindar tecnologías de gran rendimiento para el cálculo, almacenamiento y comunicaciones a través de la ciudad universitaria. También tiene un propósito fundamental de brindar una posibilidad y una experiencia amplia de las investigaciones científicas y el desarrollo tecnológico para todas aquellos que buscan hacerlo en la ciudad universitaria.
